In [2]:
#Web Scraping y filtro para Minciencias y separacion de productos

In [3]:
#pip install openpyxl

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
from scrapy.selector import Selector
import requests
from bs4 import BeautifulSoup
import warnings

In [3]:
warnings.filterwarnings('ignore')
warnings.warn('InsecureRequestWarning')
warnings.warn('Do not show this message')

In [4]:
dfgrupos = pd.read_excel("BDFisicoQuimicos.xlsx", engine='openpyxl') #, sheet_name='Sheet1'
tamaño_dfgrupos =dfgrupos.shape
url= dfgrupos["Links"]

In [5]:
url

0     https://scienti.minciencias.gov.co/gruplac/jsp...
1     https://scienti.minciencias.gov.co/gruplac/jsp...
2     https://scienti.minciencias.gov.co/gruplac/jsp...
3     https://scienti.minciencias.gov.co/gruplac/jsp...
4     https://scienti.minciencias.gov.co/gruplac/jsp...
5     https://scienti.minciencias.gov.co/gruplac/jsp...
6     https://scienti.minciencias.gov.co/gruplac/jsp...
7     https://scienti.minciencias.gov.co/gruplac/jsp...
8     https://scienti.minciencias.gov.co/gruplac/jsp...
9     https://scienti.minciencias.gov.co/gruplac/jsp...
10    https://scienti.minciencias.gov.co/gruplac/jsp...
11    https://scienti.minciencias.gov.co/gruplac/jsp...
12    https://scienti.minciencias.gov.co/gruplac/jsp...
13    https://scienti.minciencias.gov.co/gruplac/jsp...
14    https://scienti.minciencias.gov.co/gruplac/jsp...
15    https://scienti.minciencias.gov.co/gruplac/jsp...
16    https://scienti.minciencias.gov.co/gruplac/jsp...
17    https://scienti.minciencias.gov.co/gruplac

In [6]:
#Para el ejemplo 
url = url[0:5]

In [7]:
palabras_filtro = "recurso hidrico"
palabras_filtro2 = "metales"

dfcon = pd.DataFrame()
for i in range (0,tamaño_dfgrupos [0]):
    print("Procesando link  ",i," ....")
    url = dfgrupos["Links"][i]
    r = requests.get(url, verify=False)
    soup = BeautifulSoup(r.text, 'lxml')
    NomGrupo = soup.find("span", class_='celdaEncabezado')
    NombreGrupo = NomGrupo.text
    #Filtrar todas las tablas
    table = soup.find_all('table')
    df = pd.read_html(str(table))
    #Numero de de tablas 
    NumTablas = len(df)
    #CICLO 
    dfcon1 = pd.DataFrame()
    dfcon1["Index"] = [i]
    dfcon1["Nombre del Grupo"] = [NombreGrupo]
    #Tabla numero 0, datos basicos, que es diferente 
    #Tabla de datos 
    df = pd.read_html(str(table))[0]
    #Imputacion de los NAN
    df= df.fillna('Sin dato')
    #Tabla 1 
    tamaño = df.shape
    Nombre = df[0][0]
    Dato = []
    for j in range(1,tamaño[0]):
        Dato.append(df[0][j]+":  "+df[1][j]+"... ")
    Datos = " \n ".join(Dato)
    Datos
    dfcon1[Nombre]= [Datos]
    # Siguientes Tablas 1-5
    for i in range (1,6):
        df = pd.read_html(str(table))[i]
        tamaño = df.shape
        Nombre = df[0][0]
        Datos = []
        for j in range(1,tamaño[0]):
            Datos.append(df[0][j])
        Datos = " \n ".join(Datos)
        dfcon1[Nombre]= [Datos] 
    # Siguientes Tablas 6-12
    for i in range (6,13):
        df = pd.read_html(str(table))[i]
        tamaño = df.shape
        Nombre = df[0][0]
        df_filtrado = df[df[0].str.contains(palabras_filtro,case=False) & df[0].str.contains(palabras_filtro2,case=False) ] 
        Datos = []
        for j in range(0,tamaño[0]):
            try:
                Datos.append(df_filtrado[0][j])
            except:
                pass
        Datos = " \n ".join(Datos) 
        dfcon1[Nombre]= [Datos] 
    #Tabla 13 -44
    for i in range (13,45):
        df = pd.read_html(str(table))[i]
        tamaño = df.shape
        Nombre = df[0][0]
        Datos = []
        for j in range(1,tamaño[0]):
            try:
                df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]               
                Datos.append(df_filtrado[1][j])
            except:
                pass
        Datos = " \n ".join(Datos)
        dfcon1[Nombre]= [Datos]
    #Table 45 - 46
    for i in range (45,47):
        df = pd.read_html(str(table))[i]
        tamaño = df.shape
        Nombre = df[0][0]
        Datos = []
        for j in range(1,tamaño[0]-1):
            df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False) ]
            Datos.append(df[1][j])
        Datos = " \n ".join(Datos)
        dfcon1[Nombre]= [Datos]
    #Tabla 47 - 84 
    for i in range (47,84):
        df = pd.read_html(str(table))[i]
        tamaño = df.shape
        Nombre = df[0][0]
        Datos = []
        for j in range(1,tamaño[0]-1):
            try:
                df_filtrado = df[df[1].str.contains(palabras_filtro,case=False) & df[1].str.contains(palabras_filtro2,case=False)] #Case = false para que no importen mayusculas
                Datos.append(df_filtrado[1][j])
            except:
                pass
        Datos = " \n ".join(Datos)
        dfcon1[Nombre]= [Datos] 
    dfcon = pd.concat([dfcon, dfcon1])
dfcon = dfcon.set_index("Index")
dfcon

Procesando link   0  ....


KeyboardInterrupt: 

In [15]:
dfcon.to_excel("InormacionFiltrada.xlsx")

In [1]:
dfcon

NameError: name 'dfcon' is not defined

In [25]:
display(dfcon.loc[0][50:85])

Softwares                                                                                                                                 
Empresas de base tecnológica                                                                                                              
APROPIACIÓN SOCIAL Y DIVULGACIÓN PÚBLICA DE LA CIENCIA                                                                                    
Publicaciones editoriales no especializadas                                                                                               
Producciones de contenido digital - Audiovisual                                                                                           
Producciones de contenido digital - Sonoro                                                                                                
Producciones de contenido digital - Recursos gráficos                                                                                     
Divulgación Pública de la C

In [16]:
df = pd.read_excel("InormacionFiltrada.xlsx",engine='openpyxl') #sheet_name='Sheet1', 
df = df.set_index("Index")

In [17]:
df

,Nombre del Grupo,Datos básicos,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo,PRODUCCIÓN DE FORMACIÓN Y EXTENSIÓN,Programa académico de doctorado,Programa académico de maestría,Otro programa académico,...,Participación Ciudadana en Proyectos de CTI,"Producción en arte, arquitectura y diseño",ACTIVIDADES DE FORMACIÓN,Asesorías al Programa Ondas,Curso de Corta Duración Dictados,Trabajos dirigidos/turorías,ACTIVIDADES COMO EVALUADOR,Jurado/Comisiones evaluadoras de trabajo de grado,Participación en comités de evaluación,Demás trabajos
Index,,,,,,,,,,,,,,,,,,,,,
0,PRODUCCION SOSTENIBLE,Año y mes de formación: 2008 - 4... Departam...,1.- UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA...,Plan de trabajo: Desarrollo de proyectos de in...,1.- Biodiversidad y Recursos Genéticos 2.- Bio...,Nombre 1.- OSCAR EDUARDO SANCLEMENTE REYES 2.-...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Grupo de Investigación en Ingeniería de Proces...,Año y mes de formación: 2010 - 1... Departam...,1.- Hatch Indisa S.A.S. - (Avalado) 2.- Indisa...,"Plan de trabajo: Hatch S.A.S., dentro de su Un...",1.- Desarrollo tecnológico y Escalado de proce...,Nombre 1.- Enrique Posada Restrepo 2.- Gilmar ...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,79.- Demás trabajos : Determinación de altura ...
2,Grupo de Investigación y Laboratorio de Monito...,Año y mes de formación: 2019 - 8... Departam...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: El plan de trabajo está desar...,1.- Calidad del agua 2.- Calidad del aire 3.- ...,Nombre 1.- Mauricio Andrés Correa Ochoa 2.- Ca...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Calidad del Aire,Año y mes de formación: 2003 - 7... Departam...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: Plan de trabajo 2016-2018 1. ...,1.- Calidad del aire y salud 2.- Caracterizaci...,Nombre 1.- Néstor Y Rojas Roa 2.- Rodrigo JIME...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,75.- Trabajo de grado de maestría o especialid...,NaN,41.- Maestría : Modelo de la dispersión atmosf...,NaN,NaN
4,ARKADIUS,Año y mes de formación: 2004 - 4... Departam...,1.- UNIVERSIDAD DE MEDELLIN - (Avalado),Plan de trabajo: 1.\tACTIVIDADES DE INVESTIGAC...,1.- Automatizacion 2.- Educación en Ingeniería...,Nombre 1.- Jaime Alberto Echeverri Arias 2.- J...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Grupo de Investigación en Aplicación de Nuevas...,Año y mes de formación: 1999 - 6... Departam...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: El plan de trabajo está proye...,1.- Desarrollo de procesos intensificados usan...,Nombre 1.- Javier Fontalvo Alzate 2.- Alan Did...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,93.- Trabajo de grado de maestría o especialid...,NaN,NaN,NaN,NaN
6,Desarrollo de Estudios y Tecnologías Ambiental...,Año y mes de formación: 2013 - 1... Departam...,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: El Grupo de Investigacion DES...,1.- Biotecnología y Desarrollo Agroindustrial ...,Nombre 1.- Marlon José Bastidas Barranco 2.- N...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,REDAIRE - UNAL,Año y mes de formación: 2000 - 10... Departa...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: REDAIRE fue creado el 17 de j...,1.- Material Particulado en la Atmosfera 2.- M...,Nombre 1.- Carmen Elena Zapata Sánchez 2.- Jos...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,GISA,Año y mes de formación: 1999 - 1... Departam...,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: Plan de trabajo: Inicialmente...,"1.- Cambio climático, variabilidad climática y...",Nombre 1.- JHONNY ISAAC PEREZ MONTIEL 2.- Robe...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,69.- Trabajos de grado de pregrado : Determina...,NaN,NaN,NaN,NaN


In [18]:
#leer el DataFrame e imprimir cada columna como una tabla nueva

In [19]:
##Eliminar los sin dato y los vacios (:  -)

#Crear tabla de datos basicos
DatosBasicos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
for j in range(0,df.shape[0]):
    dfDatosBasidos = pd.DataFrame(columns=['Nombre del Grupo','Datos básicos'])
    ListaDatosBasicos = df["Datos básicos"][j].split("... ")
    for i in range (0,len(ListaDatosBasicos)-1):
        if i == 0:
            dfDatosBasidos.loc[len(dfDatosBasidos)] = [df["Nombre del Grupo"][j], ListaDatosBasicos[i]]
        else:
            dfDatosBasidos.loc[len(dfDatosBasidos)] = ["", ListaDatosBasicos[i]]
    #REMOVER LAS FILAS QUE NO SIRVEN 
    v1 = dfDatosBasidos.iloc[:, 1].str.contains("sin dato",case=False)
    v2 = dfDatosBasidos.iloc[:, 1].str.contains(":  -",case=False)
    for i in range (0,len(ListaDatosBasicos)-1):
        if (v1[i] == True) or (v2[i] == True):
            dfDatosBasidos = dfDatosBasidos.drop([i],axis=0)
    DatosBasicos = pd.concat([DatosBasicos, dfDatosBasidos])
DatosBasicos.to_excel("DatosBasicos.xlsx")
DatosBasicos

,Nombre del Grupo,Datos básicos
0,PRODUCCION SOSTENIBLE,Año y mes de formación: 2008 - 4
1,,Departamento - Ciudad: VALLE DEL CAUCA - PAL...
2,,Líder: OSCAR EDUARDO SANCLEMENTE REYES
3,,¿La información de este grupo se ha certifica...
5,,E-mail: oscar.sanclemente@unad.edu.co
...,...,...
5,,E-mail: lsilva8@cuc.edu.co
6,,Clasificación: A1 con vigencia hasta la pub...
7,,Área de conocimiento: Ingeniería y Tecnologí...
8,,Programa nacional de ciencia y tecnología: C...


In [20]:
# Crear tabla de informacion de los grupos 
DatosGrupo = df.iloc[:,0:6].drop(["Datos básicos"],axis=1)
DatosGrupo.to_excel("DatosGrupo.xlsx")
DatosGrupo

,Nombre del Grupo,Instituciones,Plan Estratégico,Líneas de investigación declaradas por el grupo,Integrantes del grupo
Index,,,,,
0,PRODUCCION SOSTENIBLE,1.- UNIVERSIDAD NACIONAL ABIERTA Y A DISTANCIA...,Plan de trabajo: Desarrollo de proyectos de in...,1.- Biodiversidad y Recursos Genéticos 2.- Bio...,Nombre 1.- OSCAR EDUARDO SANCLEMENTE REYES 2.-...
1,Grupo de Investigación en Ingeniería de Proces...,1.- Hatch Indisa S.A.S. - (Avalado) 2.- Indisa...,"Plan de trabajo: Hatch S.A.S., dentro de su Un...",1.- Desarrollo tecnológico y Escalado de proce...,Nombre 1.- Enrique Posada Restrepo 2.- Gilmar ...
2,Grupo de Investigación y Laboratorio de Monito...,1.- UNIVERSIDAD DE ANTIOQUIA - (Avalado),Plan de trabajo: El plan de trabajo está desar...,1.- Calidad del agua 2.- Calidad del aire 3.- ...,Nombre 1.- Mauricio Andrés Correa Ochoa 2.- Ca...
3,Calidad del Aire,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: Plan de trabajo 2016-2018 1. ...,1.- Calidad del aire y salud 2.- Caracterizaci...,Nombre 1.- Néstor Y Rojas Roa 2.- Rodrigo JIME...
4,ARKADIUS,1.- UNIVERSIDAD DE MEDELLIN - (Avalado),Plan de trabajo: 1.\tACTIVIDADES DE INVESTIGAC...,1.- Automatizacion 2.- Educación en Ingeniería...,Nombre 1.- Jaime Alberto Echeverri Arias 2.- J...
5,Grupo de Investigación en Aplicación de Nuevas...,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: El plan de trabajo está proye...,1.- Desarrollo de procesos intensificados usan...,Nombre 1.- Javier Fontalvo Alzate 2.- Alan Did...
6,Desarrollo de Estudios y Tecnologías Ambiental...,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: El Grupo de Investigacion DES...,1.- Biotecnología y Desarrollo Agroindustrial ...,Nombre 1.- Marlon José Bastidas Barranco 2.- N...
7,REDAIRE - UNAL,1.- UNIVERSIDAD NACIONAL DE COLOMBIA - (Avalado),Plan de trabajo: REDAIRE fue creado el 17 de j...,1.- Material Particulado en la Atmosfera 2.- M...,Nombre 1.- Carmen Elena Zapata Sánchez 2.- Jos...
8,GISA,1.- UNIVERSIDAD DE LA GUAJIRA - (Avalado),Plan de trabajo: Plan de trabajo: Inicialmente...,"1.- Cambio climático, variabilidad climática y...",Nombre 1.- JHONNY ISAAC PEREZ MONTIEL 2.- Robe...


In [21]:
# Crear todas las tablas en archivos diferentes. 

df1 = df.set_index("Nombre del Grupo")
for i in range(5,84):
    try:
        NombreColumna = df1.columns[i]
        Nombre = str(i)+' '+NombreColumna+".xlsx"
        dfsin_nan = df1[NombreColumna].dropna()
        try:
            if type(dfsin_nan[0]) != str():
                dfsin_nan.to_excel(Nombre)
        except:
            pass
    except:
        pass